In [1]:
import numpy as np
import pandas as pd
import re
import random
import spacy

# sparse matrix to store all unique corpus transition 
from scipy.sparse import dok_matrix

# Poems

## Preprocessing

In [2]:
poem_df = pd.read_csv('poem.csv')

In [3]:
poem_df.drop('Unnamed: 0', axis=1, inplace = True)

Preprocessing steps:
- Split based on the punctuations
- Lowercase all words

In [4]:
enter_split = poem_df['Content'][1]
enter_split

'Philosophic\nin its complex, ovoid emptiness,\na skillful pundit coined it as a sort\nof stopgap doorstop for those\nquaint equations\n\nRomans never\ndreamt of. In form completely clever\nand discrete—a mirror come unsilvered,\nloose watch face without the works,\na hollowed globe\n\nfrom tip to toe\nunbroken, it evades the grappling\nhooks of mass, tilts the thin rim of no thing,\nremains embryonic sum,\nnon-cogito.'

In [5]:
re.findall(r"[\w'-]+|[.,!?;\n]", enter_split.strip())

['Philosophic',
 '\n',
 'in',
 'its',
 'complex',
 ',',
 'ovoid',
 'emptiness',
 ',',
 '\n',
 'a',
 'skillful',
 'pundit',
 'coined',
 'it',
 'as',
 'a',
 'sort',
 '\n',
 'of',
 'stopgap',
 'doorstop',
 'for',
 'those',
 '\n',
 'quaint',
 'equations',
 '\n',
 '\n',
 'Romans',
 'never',
 '\n',
 'dreamt',
 'of',
 '.',
 'In',
 'form',
 'completely',
 'clever',
 '\n',
 'and',
 'discrete',
 'a',
 'mirror',
 'come',
 'unsilvered',
 ',',
 '\n',
 'loose',
 'watch',
 'face',
 'without',
 'the',
 'works',
 ',',
 '\n',
 'a',
 'hollowed',
 'globe',
 '\n',
 '\n',
 'from',
 'tip',
 'to',
 'toe',
 '\n',
 'unbroken',
 ',',
 'it',
 'evades',
 'the',
 'grappling',
 '\n',
 'hooks',
 'of',
 'mass',
 ',',
 'tilts',
 'the',
 'thin',
 'rim',
 'of',
 'no',
 'thing',
 ',',
 '\n',
 'remains',
 'embryonic',
 'sum',
 ',',
 '\n',
 'non-cogito',
 '.']

In [14]:
def split_punc(x):
    return re.findall(r"[\w'-]+|[.,!?;\n]", x.strip())

- The first list is preserving all words with a hypen or apostrophe
- The second list is matching all punctuations
- This regular expression will match these words (| is for or)

In [7]:
poem_df.columns

Index(['Author', 'Title', 'Poetry Foundation ID', 'Content'], dtype='object')

In [8]:
poem_df['Content'] = poem_df['Content'].apply(lambda x: x.lower())
poem_df['Content'] = poem_df['Content'].apply(lambda x: split_punc(x))
poem_df.head()

,Author,Title,Poetry Foundation ID,Content
0,Wendy Videlock,!,55489,"[dear, writers, ,, i, m, compiling, the, first..."
1,Hailey Leithauser,0,41729,"[philosophic, \n, in, its, complex, ,, ovoid, ..."
2,Jody Gladding,1-800-FEAR,57135,"[we'd, like, to, talk, with, you, about, fear,..."
3,Joseph Brodsky,1 January 1965,56736,"[the, wise, men, will, unlearn, your, name, .,..."
4,Ted Berrigan,3 Pages,51624,"[for, jack, collom, \n, 10, things, i, do, eve..."


## Train Markov Chain

### Check how many corpus we have

In [9]:
lst = poem_df['Content'][0].copy()
for i in range(1, poem_df.shape[0]):
    lst.extend(poem_df['Content'][i].copy())
corpus = set(lst)
print("Total unique words:", len(corpus))

Total unique words: 129942


In [10]:
corpus_dic = {}
for index, each_word in enumerate(corpus):
    corpus_dic[each_word] = index

In [156]:
corpus_dic

{'displacing': 0,
 'd-for': 1,
 'waggly': 2,
 "sov'ran": 3,
 'procinct': 4,
 "deck'd": 5,
 'downturn': 6,
 'flit': 7,
 'sedentarie': 8,
 'beneotan': 9,
 'nags': 10,
 'film-star': 11,
 'smokey': 12,
 'yellow-tape': 13,
 'appeers': 14,
 'loaches': 15,
 'nutmeg': 16,
 'war-friend': 17,
 'viridian': 18,
 'jeux': 19,
 'ids': 20,
 'prison-striped': 21,
 'church-porch': 22,
 'narcissi': 23,
 'infected': 24,
 'coronado': 25,
 'fingerlings': 26,
 'strata': 27,
 'donkande': 28,
 'rued': 29,
 'hetero': 30,
 "unreveal'd": 31,
 'tripling': 32,
 'arno-vale': 33,
 '3-cornered': 34,
 'vengeful': 35,
 'night-helm': 36,
 'toni': 37,
 "soul-shrin'd": 38,
 'waif': 39,
 'plaguy': 40,
 'seareth': 41,
 'sixpack': 42,
 'acknowledgment': 43,
 "owlet's": 44,
 'transitivity': 45,
 'rotes': 46,
 'recoiling': 47,
 'foists': 48,
 'lepidoptera': 49,
 'blewe': 50,
 'häftling': 51,
 'spoiled': 52,
 'drede': 53,
 'restriction': 54,
 'alonde': 55,
 '7-yr-old': 56,
 'guess': 57,
 'heate': 58,
 'sueur': 59,
 'high-wrought

Since the number of corpus we have is considered low, it might be best to just set the k=1 for now and find larger dataset for further k (k=window size of the words)

In [72]:
# initialize the sparse matrix
trans_matrix = dok_matrix((len(corpus), len(corpus)), dtype='float')

In [27]:
# update the sparse matrix to check the sequence words
def trace_seq(seq_words, trans_matrix):
    for i in range(1, len(seq_words)):
        cur_word_index = corpus_dic[seq_words[i-1]]
        next_word_index = corpus_dic[seq_words[i]]
        trans_matrix[cur_word_index, next_word_index] += 1
    return trans_matrix

In [74]:
# run the seq words matrix above
for i in range(poem_df.shape[0]):
    trans_matrix = trace_seq(poem_df['Content'][i], trans_matrix)

In [14]:
# alpha here can be used to push all zero freq words to have a probability weights
def get_next_word(word, trans_matrix, corpus_dic, lst_words, alpha=0):
    trans_vector = trans_matrix[corpus_dic[word], :].copy() + alpha
    return sample_weighted_random_word(trans_vector, lst_words)  

In [15]:
def sample_weighted_random_word(vector, lst_words):
    """Take random choice of words based on weighted probability"""
    vector = (vector/vector.sum()).toarray().flatten()
    x = random.choices(lst_words, weights = vector)
    return x

In [143]:
get_next_word('a', trans_matrix, corpus_dic, list(corpus_dic.keys()))

0.0
129942
129942


['religion']

In [119]:
def sample_weighted_word(vector):
    """This will return the given word using uniform as its random choice"""
    """Has not worked yet"""
    vector = (vector/vector.sum()).toarray()
    print(vector.shape)
    cumsum_vect = np.cumsum(vector)
    print(cumsum_vect[-4])
    random = np.random.uniform()
    x = 0
    print(len(cumsum_vect))
    while cumsum_vect[x]<random:
        x += 1
    return x

## Markov Chain for Poem

In [152]:
def markov_chain(init_word, lines):
    
    """
    Generate the poem
    Argument:
    - Initial word for the poem
    - Max number of lines    
    """
    poem = init_word
    count_line = 0
    word = init_word
    lst_words = list(corpus_dic.keys())
    punc = [',', '!', '?', '.']
    enter = '\n'
    
    while count_line<lines:
        next_word = get_next_word(word, trans_matrix, corpus_dic, lst_words)
        if next_word[0] in punc:
            poem += next_word[0]
        elif next_word[0] == enter:
            count_line += 1
            poem += enter
        else:
            poem += ' '+next_word[0]
    
    print(poem)

In [154]:
markov_chain('a', lines=1)

a single gravestone single new whisper boy reefer mound war joke pebble pale kind vote natural body bright president blessing picture sheep gift price name place bright world heart red favor'd piece lov'd sense plan part pen tornado condition constant huge mechanical fire sunny scorpion rotting sketchpad schedule shelf whimper weie mind handle frontier world fluid weather word visitation quick scrub footboard way mass name moth rusty catholic pushing terme trance heap double copy



As you can see from the result above, the first few words are quite good. It started with 'a single gravestone single new whisper boy'. Personally, I would say that adding a coma between gravestone and single might be better, but so far, we manage to generate this kind of lines by just using simple markov chain.

# Quotes

In [10]:
quotes = pd.read_csv('quotes.csv')

In [11]:
quotes.head()

,quote,author,category
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."
3,A friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love"
4,Darkness cannot drive out darkness: only light...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh..."


In [166]:
quotes['quote'][0].lower()

"i'm selfish, impatient and a little insecure. i make mistakes, i am out of control and at times hard to handle. but if you can't handle me at my worst, then you sure as hell don't deserve me at my best."

## Preprocessing

In [164]:
quotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499709 entries, 0 to 499708
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   quote     499708 non-null  object
 1   author    497956 non-null  object
 2   category  499646 non-null  object
dtypes: object(3)
memory usage: 11.4+ MB


In [167]:
quotes.isnull().sum()

quote          1
author      1753
category      63
dtype: int64

In [12]:
quotes.dropna(inplace=True)

In [15]:
quotes['quote'] = quotes['quote'].apply(lambda x: x.lower())
quotes['quote'] = quotes['quote'].apply(lambda x: split_punc(x))
quotes.head()

,quote,author,category
0,"[i'm, selfish, ,, impatient, and, a, little, i...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,"[you've, gotta, dance, like, there's, nobody, ...",William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,"[you, know, you're, in, love, when, you, can't...",Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."
3,"[a, friend, is, someone, who, knows, all, abou...",Elbert Hubbard,"friend, friendship, knowledge, love"
4,"[darkness, cannot, drive, out, darkness, only,...","Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh..."


In [16]:
quotes.reset_index(inplace = True)

### Get all unique words (corpus)

In [17]:
lst = quotes['quote'][0].copy()
for i in range(1, quotes.shape[0]):
    lst.extend(quotes['quote'][i].copy())
corpus = set(lst)
print("Total unique words:", len(corpus))

Total unique words: 215080


In [10]:
corpus_dic = {}
for index, each_word in enumerate(corpus):
    corpus_dic[each_word] = index

In [11]:
# initialize the sparse matrix
trans_matrix = dok_matrix((len(corpus), len(corpus)), dtype='float')

In [13]:
# run the seq words matrix above
for i in range(quotes.shape[0]):
    trans_matrix = trace_seq(quotes['quote'][i], trans_matrix)

## Markov chain for quotes

In [24]:
def markov_chain(init_word, length=15):
    
    """
    Generate the poem
    Argument:
    - Initial word for the poem
    - Max number of lines    
    """
    quotes = init_word
    count_words = 0
    word = init_word.split()[-1]
    lst_words = list(corpus_dic.keys())
    punc = [',', '!', '?', '.']
    
    while count_words<length:
        next_word = get_next_word(word, trans_matrix, corpus_dic, lst_words)
        if next_word[0] in punc:
            quotes += next_word[0]
        else:
            quotes += ' '+next_word[0]
            count_words += 1
    
    print(quotes)

In [20]:
markov_chain('happy', 15)

happy as man ; in ones. by. whose.,. he ending every in bright? grass, times. and


In [21]:
markov_chain('man', 15)

man. whose has who who followed who to who who. is walks earns, leapt may, to


## Try playing with the alpha randomness

In [28]:
def markov_chain(init_word, length=15, alpha=0):
    
    """
    Generate the poem
    Argument:
    - Initial word for the poem
    - Max number of lines    
    """
    quotes = init_word
    count_words = 0
    word = init_word.split()[-1]
    lst_words = list(corpus_dic.keys())
    punc = [',', '!', '?', '.']
    
    while count_words<length:
        next_word = get_next_word(word, trans_matrix, corpus_dic, lst_words, alpha)
        if next_word[0] in punc:
            quotes += next_word[0]
        else:
            quotes += ' '+next_word[0]
            count_words += 1
    
    print(quotes)

In [29]:
markov_chain('happy', 15, alpha=1)

happy minuses umiibig short-tempered actionthat --perhaps restating notes-ah omission hote salvaged-wood tableaus forests way-if self-disciplinesupervene arrivalbut


In [30]:
markov_chain('man', 15, alpha=1)

man speakwhen dehlin lucianus ottuagenario dopest junctions, whitchurch come-uppance eulogize carradoon bcom hemorrhaged can hyperventilated witheach


# Adding preposition

In [3]:
nlp = spacy.load('en_core_web_sm')

In [7]:
doc = nlp("man speakwhen dehlin lucianus ottuagenario dopest junctions, whitchurch come-uppance eulogize carradoon bcom hemorrhaged can hyperventilated witheach")

In [8]:
doc[0].pos_

'PROPN'

## Extracting the preposition

In [36]:
test_str = " ".join(quotes['quote'][0])
doc = nlp(test_str)
# iterate for each word and get the preposition
print([i.pos_ for i in doc])

['PRON', 'AUX', 'ADJ', 'PUNCT', 'ADJ', 'CCONJ', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'SCONJ', 'ADP', 'NOUN', 'CCONJ', 'ADP', 'NOUN', 'ADV', 'PART', 'VERB', 'PUNCT', 'CCONJ', 'SCONJ', 'PRON', 'VERB', 'PART', 'VERB', 'PRON', 'ADP', 'DET', 'ADJ', 'PUNCT', 'ADV', 'PRON', 'ADV', 'SCONJ', 'PROPN', 'AUX', 'PART', 'VERB', 'PRON', 'ADP', 'DET', 'ADJ', 'PUNCT']


In [33]:
def rec_prep(x):
    x = nlp(x)
    return [i.pos_ for i in x]

In [37]:
quotes['prep'] = quotes['quote'].apply(lambda x: " ".join(x))

In [39]:
quotes['prep'] = quotes['prep'].apply(lambda x: rec_prep(x))

In [44]:
quotes.drop('index', axis=1, inplace = True)

In [46]:
quotes['quote'] = quotes['quote'].apply(lambda x: " ".join(x))
quotes['prep'] = quotes['prep'].apply(lambda x: " ".join(x))

In [48]:
quotes.to_csv('quotes_with_prep.csv', index = False)

## Use the new quotes dataframe

In [2]:
quotes_df = pd.read_csv('quotes_with_prep.csv')

In [7]:
quotes_df.head()

,quote,author,category,prep
0,"i'm selfish , impatient and a little insecure ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak...",PRON AUX ADJ PUNCT ADJ CCONJ DET ADJ NOUN PUNC...
1,you've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love...",PRON AUX VERB PART VERB INTJ PRON AUX PRON VER...
2,you know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s...",PRON VERB PRON AUX ADP NOUN ADV PRON VERB PART...
3,a friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love",DET NOUN AUX PRON PRON VERB ADV ADP PRON CCONJ...
4,darkness cannot drive out darkness only light ...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh...",NOUN VERB PART VERB ADP NOUN ADV NOUN VERB AUX...


In [8]:
quotes_df.isnull().sum()

quote       1
author      0
category    0
prep        1
dtype: int64

In [9]:
quotes_df.dropna(inplace = True)
quotes_df.reset_index(inplace = True)

In [11]:
quotes_df.drop('index', axis=1, inplace = True)

### Calculate the preposition corpus

In [12]:
lst = quotes_df['prep'][0].split()
for i in range(1, quotes_df.shape[0]):
    lst.extend(quotes_df['prep'][i].split())
corpus_prep = set(lst)
print("Total unique words:", len(corpus_prep))

Total unique words: 18


In [14]:
corpus_prep

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SPACE',
 'SYM',
 'VERB',
 'X'}

In [20]:
corpus_prep_dic = {}
for index, each_prep in enumerate(corpus_prep):
    corpus_prep_dic[each_prep] = index

In [25]:
prep_trans_matrix = np.zeros((len(corpus_prep_dic), len(corpus_prep_dic)), dtype=float)

In [28]:
# update the sparse matrix to check the sequence words
def trace_seq(seq_words, trans_matrix, corpus_dic):
    for i in range(1, len(seq_words)):
        cur_word_index = corpus_dic[seq_words[i-1]]
        next_word_index = corpus_dic[seq_words[i]]
        trans_matrix[cur_word_index, next_word_index] += 1
    return trans_matrix

In [30]:
# run the seq words matrix above
for i in range(quotes_df.shape[0]):
    prep_trans_matrix = trace_seq(quotes_df['prep'][i].split(), prep_trans_matrix, corpus_prep_dic)

### Calculate the word corpus

In [18]:
lst = quotes_df['quote'][0].split()
for i in range(1, quotes_df.shape[0]):
    lst.extend(quotes_df['quote'][i].split())
corpus = set(lst)
print("Total unique words:", len(corpus))

Total unique words: 215079


In [22]:
corpus_dic = {}
for index, each_word in enumerate(corpus):
    corpus_dic[each_word] = index

In [26]:
# initialize the sparse matrix
trans_matrix = dok_matrix((len(corpus), len(corpus)), dtype='float')

In [32]:
# run the seq words matrix above
for i in range(quotes_df.shape[0]):
    trans_matrix = trace_seq(quotes_df['quote'][i].split(), trans_matrix, corpus_dic)

### Record each unique words in each prepositions

In [34]:
quotes_df.head()

,quote,author,category,prep
0,"i'm selfish , impatient and a little insecure ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak...",PRON AUX ADJ PUNCT ADJ CCONJ DET ADJ NOUN PUNC...
1,you've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love...",PRON AUX VERB PART VERB INTJ PRON AUX PRON VER...
2,you know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s...",PRON VERB PRON AUX ADP NOUN ADV PRON VERB PART...
3,a friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love",DET NOUN AUX PRON PRON VERB ADV ADP PRON CCONJ...
4,darkness cannot drive out darkness only light ...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh...",NOUN VERB PART VERB ADP NOUN ADV NOUN VERB AUX...


In [42]:
dic_word_prep = {}
for i in corpus_prep_dic:
    dic_word_prep[i] = []

In [38]:
for i in range(len(quotes_df['quote'][0].split())):
    if quotes_df['quote'][0].split()[i] not in dic_word_prep[quotes_df['prep'][0].split()[i]]:
        dic_word_prep[quotes_df['prep'][0].split()[i]].append(quotes_df['quote'][0].split()[i])

In [39]:
dic_word_prep

{'PRON': ["i'm", 'make', 'am', "can't", 'my', 'as', 'best'],
 'NOUN': ['.', ',', 'and', 'hard'],
 'SPACE': [],
 'INTJ': [],
 'NUM': [],
 'VERB': ['mistakes', '.', 'handle', 'at', 'my'],
 'X': [],
 'ADJ': [',', 'and', 'insecure', 'then'],
 'PUNCT': ['impatient', 'i', 'but', 'you'],
 'CCONJ': ['a', 'at', 'if'],
 'PROPN': ['deserve'],
 'ADV': ['to', 'sure', 'hell'],
 'PART': ['handle', 'me', 'at'],
 'SYM': [],
 'AUX': ['selfish', 'out', 'me'],
 'DET': ['little', ','],
 'ADP': ['control', 'times', 'worst', '.'],
 'SCONJ': ['of', 'you', "don't"]}

In [41]:
quotes_df['quote'] = quotes_df['quote'].apply(lambda x: x.split())
quotes_df['prep'] = quotes_df['prep'].apply(lambda x: x.split())

In [49]:
def rec_prep_word(df):
    for _, row in df.iterrows():
        for i in range(len(row[0])):
            if row[0][i] not in dic_word_prep[row[3][i]]:
                dic_word_prep[row[3][i]].append(row[0][i])
    return dic_word_prep

In [50]:
dic_word_prep = rec_prep_word(quotes_df)

### Generate the Markov Chain

In [56]:
# alpha here can be used to push all zero freq words to have a probability weights
def get_next_prep(prep, prep_trans_matrix, corpus_prep_dic, lst_corpus_prep):
    vector_prep = prep_trans_matrix[corpus_prep_dic[prep], :].flatten()
    return lst_corpus_prep[np.argmax(vector_prep)]

In [57]:
# alpha here can be used to push all zero freq words to have a probability weights
def get_next_word(word, trans_matrix, corpus_dic, next_prep, alpha=0):
    lst_words = dic_word_prep[next_prep]
    trans_vector = trans_matrix[corpus_dic[word], get_index(lst_words, corpus_dic)].copy() + alpha
    return sample_weighted_random_word(trans_vector, lst_words) 

def get_index(words, corpus_dic):
    lst_index = []
    for i in words:
        lst_index.append(corpus_dic[i])
    return lst_index

In [58]:
def sample_weighted_random_word(vector, lst_words):
    """Take random choice of words based on weighted probability"""
    vector = (vector/vector.sum()).toarray().flatten()
    x = random.choices(lst_words, weights = vector)
    return x

In [99]:
def generate_quotes(init_word, length, alpha=0):
    
    """
    This is a function to generate quotes using the hidden markov model procedure.
    The procedure consist of 2 steps:
    - identifying the preposition
    - get the next word based on the preposition chosen
    """
    
    quotes = init_word
    count_words = 0
    punc = [',', '!', '?', '.']
    lst_prep = list(corpus_prep_dic.keys())
    word = init_word[-1]
    
    # iterate until max length reached
    while count_words<length:
        
        # get the current preposition context
        context = nlp(quotes)[-1].pos_

        # get the new word and its preposition
        next_prep = get_next_prep(context, prep_trans_matrix, corpus_prep_dic, lst_prep)
        next_word = get_next_word(word, trans_matrix, corpus_dic, next_prep, alpha)
        
        if next_word[0] in punc:
            quotes += next_word[0]
        else:
            quotes += ' '+next_word[0]
            count_words += 1
            
        # get the current word 
        word = next_word[0]
        
    print(quotes)
        
    

In [97]:
nlp = spacy.load('en_core_web_sm')

In [107]:
generate_quotes('happy', 15)

happy, artemis met the revolution is pleasure, killing rats and maybe all true to assign myself


In [109]:
generate_quotes('man', 15)

man the fact need to good short of the main ballroom dance between our failure is


In [113]:
generate_quotes('beauty', 15)

beauty el silencio de janeiro, this thinking for small cost to spend their views. and at


The result shows that we may be able to generate a better quotes using the help of knowing the preposition context of the words. However, the quote result is still quite random. Thus, we may have to use a more powerful model, as we lack the domain knowledge of language. Henceforth, deep learning models may be quite handy for this kind of problems (LSTM many-to-many). But, this model is beyond the scope of this project.